In [1]:
import pandas as pd
import wikipedia as wp

In [2]:
results = pd.read_excel('Results_Rio_2016.xlsx')

In [3]:
def reduce_string(string):
    return string.replace('\n', ' ').replace('\r', '').replace("==", "").replace("=", "")

In [4]:
def count(string):
    counter = 0
    saw_space = False
    for char in string:
        if char == " ":
            if not saw_space:
                counter += 1
            saw_space = True
        else:
            saw_space = False
    return counter

In [5]:
def count_words(title):
    wordcount = []
    content = wp.page(title).content
    wordcount.append(count(reduce_string(content)))
    return wordcount

In [6]:
def add_name_column(Women, Men):
    Women['Wiki_Count'] = -1
    Women['Names'] = Women['Names'].str.split(', ').str[1] + ' ' + Women['Names'].str.split(', ').str[0]
    
    Men['Wiki_Count'] = -1
    Men['Names'] = Men['Names'].str.split(', ').str[1] + ' ' + Men['Names'].str.split(', ').str[0]
    return Women, Men

In [7]:
def filter_event(discipline, event, phase):    
    Women = results.loc[(results['Discipline'] == 'Diving')
                             & (results['Event'] == '10m platform') 
                             & (results['Gender'] == 'Women') 
                             & (results['Phase'] == 'Final Ranking')] 

    Women = pd.DataFrame(Women)
    Women.reset_index(drop=True, inplace = True)

    Men = results.loc[(results['Discipline'] == discipline)
                             & (results['Event'] == event) 
                             & (results['Gender'] == 'Men') 
                             & (results['Phase'] == phase)] 

    Men = pd.DataFrame(Men)
    Men.reset_index(drop=True, inplace = True)
    
    Women, Men = add_name_column(Women, Men)
    return Women, Men


In [8]:
def get_wikipedia_count(Event, keyword):
    for x in range(len(Event.index)):        
        name = Event['Names'].values[x]
        try:
            Event.loc[x,'Wiki_Count'] = count_words(name)
        except wp.exceptions.DisambiguationError as de:
            try:
                name = name + ' ({})'.format(keyword)
                Event.loc[x,'Wiki_Count'] = count_words(name)
            except wp.exceptions.PageError as pe:
                print('Error')    
        except wp.exceptions.PageError as pe:
            print('Error')
    return Event      


In [9]:
Diving_10m_Women, Diving_10m_Men = filter_event('Diving', '10m platform', 'Final Ranking')
Diving_10m_Women.reset_index(drop=True, inplace=True)

In [10]:
get_wikipedia_count(Diving_10m_Women, 'diver')
get_wikipedia_count(Diving_10m_Men, 'diver')

/anaconda3/envs/ds_env/lib/python3.6/site-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /anaconda3/envs/ds_env/lib/python3.6/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Error
Error


,Year,Sport,Discipline,Event,EventGender,Phase,PhaseXX,Unit,UnitXX,Names,Gender,NOC,Teammembers,Rank,Results,Medal,Adversary,Adversary NOC,Wiki_Count
0,2016,Aquatics,Diving,10m platform,M,Final Ranking,0,Final ranking,00,Aisen CHEN,Men,CHN,NaN,1.0,585.30,Gold,NaN,NaN,513
1,2016,Aquatics,Diving,10m platform,M,Final Ranking,0,Final ranking,00,German SANCHEZ,Men,MEX,NaN,2.0,532.70,Silver,NaN,NaN,793
2,2016,Aquatics,Diving,10m platform,M,Final Ranking,0,Final ranking,00,David BOUDIA,Men,USA,NaN,3.0,525.25,Bronze,NaN,NaN,813
3,2016,Aquatics,Diving,10m platform,M,Final Ranking,0,Final ranking,00,Benjamin AUFFRET,Men,FRA,NaN,4.0,507.35,NaN,NaN,NaN,57
4,2016,Aquatics,Diving,10m platform,M,Final Ranking,0,Final ranking,00,Martin WOLFRAM,Men,GER,NaN,5.0,492.90,NaN,NaN,NaN,104
5,2016,Aquatics,Diving,10m platform,M,Final Ranking,0,Final ranking,00,Bo QIU,Men,CHN,NaN,6.0,488.20,NaN,NaN,NaN,573
6,2016,Aquatics,Diving,10m platform,M,Final Ranking,0,Final ranking,00,Rafael QUINTERO,Men,PUR,NaN,7.0,485.35,NaN,NaN,NaN,289
7,2016,Aquatics,Diving,10m platform,M,Final Ranking,0,Final ranking,00,Victor MINIBAEV,Men,RUS,NaN,8.0,481.60,NaN,NaN,NaN,30
8,2016,Aquatics,Diving,10m platform,M,Final Ranking,0,Final ranking,00,Sascha KLEIN,Men,GER,NaN,9.0,424.15,NaN,NaN,NaN,83
9,2016,Aquatics,Diving,10m platform,M,Final Ranking,0,Final ranking,00,Ivan GARCIA,Men,MEX,NaN,10.0,418.95,NaN,NaN,NaN,508
